# Chapter 6 - Managed Agents: Autonomous AI Agents with Strands Agents SDK 


## Overview
This notebook demonstrates how to build autonomous AI agents using Strands, a framework that enables creating AI agents with both custom and built-in tools. Strands provides a powerful way to build agents that can perform complex tasks by combining custom business logic with pre-built tools.

In this example, we'll create a customer support agent that can:
- Check order status using custom business logic
- Perform calculations using built-in tools
- Handle complex customer inquiries autonomously

## What You'll Learn
- How to set up Strands with Amazon Bedrock models
- Creating custom tools for business-specific functionality
- Using built-in tools for common tasks
- Building autonomous agents that can reason and execute complex workflows


## 1. Setup - Install Required Packages
!pip install -r requirements.txt

In [ ]:
# Install Strands and required packages
%pip install strands strands-tools


In [ ]:
# Import Strands components
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel


## 2. Configure Bedrock Model


In [ ]:
# Configure Bedrock model for Strands
bedrock_model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",  # Use the model you want to use. See https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html for more information
    temperature=0.3,
    streaming=True,  # Enable/disable streaming
)

print("Bedrock model configured successfully!")


## 3. Create Custom Tools for Business Logic


In [ ]:
# Sample order database (in production, this would be a real database)
order_database = {
    "ORD12345": {
        "customer_name": "John Doe",
        "item": "wireless headphones",
        "status": "Processing",
        "order_date": "2024-01-15",
        "estimated_delivery": "2024-01-20",
        "total_amount": 99.99
    },
    "ORD67890": {
        "customer_name": "Jane Smith",
        "item": "smart watch",
        "status": "Shipped",
        "order_date": "2024-01-10",
        "estimated_delivery": "2024-01-18",
        "total_amount": 249.99
    },
    "ORD11111": {
        "customer_name": "Rohit Kumar",
        "item": "laptop",
        "status": "Delivered",
        "order_date": "2024-01-05",
        "delivery_date": "2024-01-12",
        "total_amount": 1299.99
    }
}


In [ ]:
@tool
def check_order_status(order_id: str) -> str:
    """Check the status of a customer order"""
    order = order_database.get(order_id.upper())
    if order:
        status_info = f"Order {order_id} for {order['customer_name']}:"
        status_info += f"\n- Item: {order['item']}"
        status_info += f"\n- Status: {order['status']}"
        status_info += f"\n- Order Date: {order['order_date']}"
        
        if order['status'] == 'Delivered':
            status_info += f"\n- Delivered on: {order['delivery_date']}"
        else:
            status_info += f"\n- Estimated Delivery: {order['estimated_delivery']}"
            
        status_info += f"\n- Total Amount: ${order['total_amount']}"
        return status_info
    else:
        return f"Order {order_id} not found. Please check the order ID and try again."


In [ ]:
@tool
def calculate_refund_amount(order_id: str, quantity: int, unit_price: float) -> str:
    """Calculate refund amount for returned items"""
    order = order_database.get(order_id.upper())
    if not order:
        return f"Order {order_id} not found."
    
    total_refund = quantity * unit_price
    return f"Refund calculation for order {order_id}:\n" \
           f"- Quantity: {quantity} items\n" \
           f"- Unit Price: ${unit_price}\n" \
           f"- Total Refund: ${total_refund:.2f}"


In [ ]:
@tool
def get_product_info(product_name: str) -> str:
    """Get detailed product information"""
    # Sample product database
    products = {
        "wireless headphones": {
            "price": 99.99,
            "description": "High-quality wireless headphones with noise cancellation",
            "availability": "In Stock",
            "warranty": "1 year"
        },
        "smart watch": {
            "price": 249.99,
            "description": "Advanced smartwatch with health monitoring and GPS",
            "availability": "In Stock",
            "warranty": "2 years"
        },
        "laptop": {
            "price": 1299.99,
            "description": "High-performance laptop with 16GB RAM and 512GB SSD",
            "availability": "Limited Stock",
            "warranty": "3 years"
        }
    }
    
    product = products.get(product_name.lower())
    if product:
        return f"Product: {product_name.title()}\n" \
               f"Price: ${product['price']}\n" \
               f"Description: {product['description']}\n" \
               f"Availability: {product['availability']}\n" \
               f"Warranty: {product['warranty']}"
    else:
        return f"Product '{product_name}' not found in our catalog."


## 4. Create the Autonomous Agent


In [ ]:
# Create agent with both custom and built-in tools
agent = Agent(
    model=bedrock_model,
    system_prompt="You are a helpful customer support agent that assists customers with orders, product information, and calculations. You can check order status, calculate refunds, provide product details, and perform mathematical calculations. Always be friendly and professional.",
    tools=[check_order_status, calculate_refund_amount, get_product_info, calculator]
)

print("Autonomous agent created successfully with custom and built-in tools!")


## 5. Test the Agent with Various Scenarios


In [ ]:
# Test 1: Check order status using custom tool
print("=== Test 1: Order Status Check ===")
response1 = agent("What's the status of my order ORD12345?")
print(response1)
print("\n" + "="*50 + "\n")


In [ ]:
# Test 2: Use built-in calculator tool
print("=== Test 2: Built-in Calculator ===")
response2 = agent("If I return 3 items at $25.99 each, what's my total refund?")
print(response2)
print("\n" + "="*50 + "\n")


In [ ]:
# Test 3: Get product information
print("=== Test 3: Product Information ===")
response3 = agent("Can you tell me about the wireless headphones?")
print(response3)
print("\n" + "="*50 + "\n")


In [ ]:
# Test 4: Complex scenario combining multiple tools
print("=== Test 4: Complex Multi-Tool Scenario ===")
response4 = agent("I want to return 2 wireless headphones from my order ORD12345. Each headphone costs $99.99. Can you check my order status and calculate the refund amount?")
print(response4)
print("\n" + "="*50 + "\n")


In [ ]:
# Test 5: Mathematical calculation with context
print("=== Test 5: Mathematical Calculation with Context ===")
response5 = agent("I have an order for $1299.99 and I want to apply a 10% discount. What would be the final amount after discount?")
print(response5)
print("\n" + "="*50 + "\n")


## 6. Key Benefits of Strands

### Advantages of Using Strands:

1. **Autonomous Operation**: Agents can reason, plan, and execute complex workflows without human intervention

2. **Tool Integration**: Seamlessly combine custom business logic with built-in tools

3. **Streaming Support**: Real-time responses for better user experience

4. **Flexible Architecture**: Easy to add new tools and capabilities

5. **Bedrock Integration**: Leverage AWS Bedrock's powerful foundation models

6. **Error Handling**: Built-in error handling and edge case management

### Use Cases:
- Customer support automation
- Data analysis and reporting
- Content generation with calculations
- Multi-step workflow automation
- Interactive applications with AI reasoning


## 7. Best Practices

### When Building Strands Agents:

1. **Clear Tool Descriptions**: Provide detailed docstrings for custom tools

2. **Error Handling**: Implement proper error handling in custom tools

3. **System Prompts**: Craft clear system prompts that define the agent's role and capabilities

4. **Tool Selection**: Choose appropriate tools for the task - custom for business logic, built-in for common operations

5. **Testing**: Test agents with various scenarios including edge cases

6. **Monitoring**: Monitor agent performance and tool usage

7. **Security**: Ensure proper access controls for custom tools that interact with sensitive data
